# Pandas (2)

Материалы:
* Макрушин С.В. "Лекция 2: Библиотека Pandas"
* https://pandas.pydata.org/docs/user_guide/index.html#
* https://pandas.pydata.org/docs/reference/index.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Загрузите данные из файла `sp500hst.txt` и обозначьте столбцы в соответствии с содержимым: `"date", "ticker", "open", "high", "low", "close", "volume"`.

In [1]:
import pandas as pd
df = pd.read_csv("02_pandas_data/sp500hst.txt", sep=",", header=None,
                 names=["date", "ticker", "open", "high", "low", "close", "volume"])
df.head(2)

,date,ticker,open,high,low,close,volume
0,20090821,A,25.60,25.61,25.22,25.55,34758
1,20090824,A,25.64,25.74,25.33,25.50,22247


2. Посчитайте количество уникальных цифр, которые используются каждой строке в столбце volume.

In [3]:
def n_digits(volume):
    return len(set(str(volume)))

# n_digits(34758), n_digits(22247)

df["volume"].map(n_digits)

0         5
1         3
2         5
3         2
4         4
         ..
122569    4
122570    5
122571    5
122572    5
122573    4
Name: volume, Length: 122574, dtype: int64

3. Для каждой строки рассчитайте разность между значениями high и low, если индекс столбца нечетный, и разность между close и high в противном случае.

In [4]:
def diff(row):
    if row.name % 2:
        return row["high"] - row["low"]
    else:
        return row["close"] - row["high"]

df.apply(diff, axis=1)

0        -0.0600
1         0.4100
2        -0.3600
3         0.4975
4        -0.0300
           ...  
122569    0.5200
122570   -0.4700
122571    0.7100
122572   -1.4100
122573    1.0700
Length: 122574, dtype: float64

In [5]:
import numpy as np

np.where(
    df.index % 2 == 1,
    df['high'] - df['low'],
    df['close'] - df['high']
)

array([-0.06,  0.41, -0.36, ...,  0.71, -1.41,  1.07])

4. Рассчитайте суммарный объем торгов для для одинаковых значений тикеров.

In [6]:
df.groupby('ticker')["volume"].sum()

ticker
A        8609336
AA      81898998
AAPL    52261170
ABC      9006756
ABT     18975870
          ...   
XTO     21297931
YHOO    56837171
YUM     10971538
ZION    15551119
ZMH      4938916
Name: volume, Length: 524, dtype: int64

5. Загрузите данные из файла sp500hst.txt и обозначьте столбцы в соответствии с содержимым: "date", "ticker", "open", "high", "low", "close", "volume". Добавьте столбец с расшифровкой названия тикера, используя данные из файла `sp_data2.csv` . В случае нехватки данных об именах тикеров корректно обработать их.

In [8]:
df2 = pd.read_csv('02_pandas_data/sp_data2.csv', sep=';', names=['ticker', 'name', '%'])
df2.head()

,ticker,name,%
0,AAPL,Apple,3.6%
1,AMZN,Amazon.com,3.2%
2,GOOGL,Alphabet,3.1%
3,GOOG,Alphabet,3.1%
4,MSFT,Microsoft,3.0%


## Лабораторная работа №2.2

__Данная работа является продолжением ЛР №2. Для начала работы загрузите таблицы (см. задание 1.1)__

In [2]:
import pandas as pd

In [3]:
recipes = pd.read_csv('02_pandas_data/recipes_sample.csv')
reviews = pd.read_csv('02_pandas_data/reviews_sample.csv', index_col = 0)

In [17]:
recipes.head(3)

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0


### Применение функций к pd.Series и pd.DataFrame

4.1 Напишите функцию, которая переводит минуты в формат "XhYm". Примените эту функцию к столбцу `minutes` таблицы `recipes` (без перезаписи исходного столбца) при помощи метода `map`.

In [4]:
def m_to_hm(m: int) -> str:
    return f'{m//60}h{m%60}m'

assert m_to_hm(90) == "1h30m"
assert m_to_hm(10) == "0h10m"

In [5]:
recipes.minutes.map(m_to_hm)

0        1h30m
1        0h10m
2        0h30m
3        0h45m
4        0h25m
         ...  
29995    1h20m
29996     4h0m
29997    1h15m
29998     1h0m
29999    0h29m
Name: minutes, Length: 30000, dtype: object

In [6]:
recipes.minutes

0         90
1         10
2         30
3         45
4         25
        ... 
29995     80
29996    240
29997     75
29998     60
29999     29
Name: minutes, Length: 30000, dtype: int64

4.2 На основе таблицы `recipes` создайте таблицу, которая содержит только текстовые столбцы (используйте метод `select_dtypes`).  Примените к каждому элементу этой таблицы строковый метод `str.capitalize` при помощи метода `applymap`, не удаляя пропуски.

In [12]:
only_object = recipes.select_dtypes(include='object')

In [13]:
only_object = only_object.applymap(str.capitalize, na_action='ignore')

In [14]:
only_object.head(2)

,name,submitted,description
0,George s at the cove black bean soup,2002-10-25,An original recipe created by chef scott meska...
1,Healthy for them yogurt popsicles,2003-07-26,My children and their friends ask for my homem...


4.3 Напишите функцию, которая принимает на вход серию `pd.Series` и для серий, содержащих текстовые данные, возвращает максимальную длину строк в ней, а для числовых серий возвращает минимальный элемент в этой серии. Примените данную функцию к каждому столбцу таблицы `recipes` при помощи метода `apply`.

In [31]:
def get_stats(x: pd.Series) -> int:
    if x.dtype == 'object':
        return max(x.dropna().str.len())
    else:
        return min(x.dropna())
    

assert get_stats(pd.Series(['a', 'bbbb', 'ccc', None])) == 4
assert get_stats(pd.Series([1, 3, 2, None])) == 1


In [29]:
recipes.apply(get_stats, axis = 0)

name                83.0
id                  48.0
minutes              0.0
contributor_id    1530.0
submitted           10.0
n_steps              1.0
description       6321.0
n_ingredients        1.0
dtype: float64

### Группировки таблиц `pd.DataFrame`

5.1 Посчитайте количество рецептов, представленных каждым из участников (`contributor_id`). Какой участник добавил максимальное кол-во рецептов?

In [44]:
recipes.contributor_id.value_counts()
# Вероятно 89831

89831      421
37449      346
37779      345
1533       186
169430     183
          ... 
1061628      1
1076183      1
429061       1
64032        1
186118       1
Name: contributor_id, Length: 8404, dtype: int64

5.2 Посчитайте средний рейтинг к каждому из рецептов. Для скольких рецептов отсутствуют отзывы? Обратите внимание, что отзыв с нулевым рейтингом или не заполненным текстовым описанием не считается отсутствующим.

In [53]:
reviews[['recipe_id', 'rating']].groupby('recipe_id').mean('rating')

,rating
recipe_id,
48,1.000000
55,4.750000
66,4.944444
91,4.750000
94,5.000000
...,...
536547,5.000000
536610,0.000000
536728,4.000000


5.3 Посчитайте количество рецептов с разбивкой по годам создания.

In [58]:
recipes.submitted.map(lambda x: x[:4]).value_counts()

2007    4429
2008    4029
2006    3473
2005    3130
2009    2963
2002    2644
2003    2334
2004    2153
2010    1538
2011     922
2012     659
2001     589
2013     490
1999     275
2014     139
2000     104
2015      42
2017      39
2016      24
2018      24
Name: submitted, dtype: int64

5.4 Напишите функцию, которая принимает на вход таблицу (аналогичную `recipes` по набору столбцов), и возвращает `True` в том случае, если в столбце `minutes` присутствуют только значения, меньшие либо равные 10. Сгруппируйте таблицу `recipes` по полю `contributor_id` и для каждого участника выясните, справедливо ли, что все его рецепты занимают не более 10 минут.

In [61]:
import numpy as np

In [68]:
def has_only_fast_recipes(x: pd.DataFrame) -> bool:
    if sum(x.minutes <= 10) == len(x):
        return True
    else:
        return False

assert not has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 90, 1: 10},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)
assert has_only_fast_recipes(
    pd.DataFrame(
        {
            "name": {0: "george s", 1: "healthy"},
            "id": {0: 44123, 1: 67664},
            "minutes": {0: 7, 1: 5},
            "contributor_id": {0: 35193, 1: 91970},
            "submitted": {0: "2002-10-25", 1: "2003-07-26"},
            "n_steps": {0: np.nan, 1: np.nan},
            "description": {0: "123", 1: "zxc"},
            "n_ingredients": {0: 18.0, 1: np.nan},
        }
    )
)

In [76]:
df = recipes[['contributor_id', 'minutes']].groupby('contributor_id').max('minutes')
# Все кто больше 10
df[df.minutes > 10]

,minutes
contributor_id,
1530,50
1533,2890
1534,17340
1535,630
1538,125
...,...
2001968497,210
2002059754,50
2002234079,45


In [77]:
# все кто меньше 10 минут
df[df.minutes <= 10]

,minutes
contributor_id,
1567,0
1573,0
1603,0
1648,10
1654,10
...,...
1800069055,2
2000034751,8
2001051242,10


### Объединение таблиц `pd.DataFrame`

6.1 При помощи объединения таблиц, создайте `DataFrame`, состоящий из четырех столбцов: `id`, `name`, `user_id`, `rating`. Рецепты, на которые не оставлен ни один отзыв, должны отсутствовать в полученной таблице. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и попытавшись найти строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [167]:
from turtle import right


df = recipes[['id', 'name']].merge(reviews[['user_id', 'rating', 'recipe_id']], 
                              how = 'right',
                              left_on='id',
                              right_on = 'recipe_id')
del df['recipe_id']
df

,id,name,user_id,rating
0,57993,salmon with tomatoes,21752,5
1,142201,lemon cajun stir fry,431813,5
2,252013,cottage cheese honey and cinnamon on toast,400708,4
3,404716,belly buttons,2001852463,5
4,129396,double tomato soup,95810,5
...,...,...,...,...
126691,335534,melissa s lemon pepper chicken,1270706,4
126692,8701,should be illegal oven bbq ribs,2282344,0
126693,222001,cheesy bacon bread,689540,5
126694,354979,kittencal s pan seared steak stove top to ove...,2000242659,5


In [168]:
df[df['id'] == 401411]

,id,name,user_id,rating


6.2 При помощи объединения таблиц и группировок, создайте `DataFrame`, состоящий из трех столбцов: `recipe_id`, `name`, `review_count`, где столбец `review_count` содержит кол-во отзывов, оставленных на рецепт `recipe_id`. У рецептов, на которые не оставлен ни один отзыв, в столбце `review_count` должен быть указан 0. Подтвердите правильность работы вашего кода, выбрав рецепт, не имеющий отзывов, и найдя строку, соответствующую этому рецепту, в полученном `DataFrame`.

In [164]:
df = reviews[['recipe_id']].value_counts()
df = df.rename_axis('recipe_id').reset_index()
df['review_count'] = df[0]
del df[0]

df = recipes[['name','id']].merge(df,
                            how = 'left',
                            left_on= 'id',
                            right_on='recipe_id',
                            ).fillna(0)
del df['recipe_id']

df[df['id'] == 401411]

,name,id,review_count
8828,crunchy ranch croutons,401411,0.0


6.3. Выясните, рецепты, добавленные в каком году, имеют наименьший средний рейтинг?

In [173]:
df = reviews[['recipe_id', 'rating']].merge(recipes[['id', 'submitted']],
                                        how = 'left',
                                        left_on = 'recipe_id',
                                        right_on = 'id')[['rating', 'submitted']]

In [174]:
df['Year'] = df.submitted.map(lambda x: x[:4])
del df['submitted']

In [176]:
df.groupby('Year').mean('rating')

,rating
Year,
1999,4.274895
2000,4.284585
2001,4.393945
2002,4.404645
2003,4.439152
2004,4.463778
2005,4.409687
2006,4.416310
2007,4.420073


In [ ]:
# 2018

### Сохранение таблиц `pd.DataFrame`

7.1 Отсортируйте результат выполнения задания 6.1 в порядке убывания величины столбца `id` и сохраните результаты в csv файл. 

In [181]:
from turtle import right


df = recipes[['id', 'name']].merge(reviews[['user_id', 'rating', 'recipe_id']], 
                              how = 'right',
                              left_on='id',
                              right_on = 'recipe_id')
del df['recipe_id']
df.sort_values('id').to_csv('7.csv')

7.2 Воспользовавшись `pd.ExcelWriter`, cохраните результаты 6.1 и 6.2 в файл: на лист с названием `Рецепты с оценками` сохраните результаты выполнения 6.1; на лист с названием `Количество отзывов по рецептам` сохраните результаты выполнения 6.2.

In [182]:
from turtle import right


df1 = recipes[['id', 'name']].merge(reviews[['user_id', 'rating', 'recipe_id']], 
                              how = 'right',
                              left_on='id',
                              right_on = 'recipe_id')
del df1['recipe_id']
df1

,id,name,user_id,rating
0,57993,salmon with tomatoes,21752,5
1,142201,lemon cajun stir fry,431813,5
2,252013,cottage cheese honey and cinnamon on toast,400708,4
3,404716,belly buttons,2001852463,5
4,129396,double tomato soup,95810,5
...,...,...,...,...
126691,335534,melissa s lemon pepper chicken,1270706,4
126692,8701,should be illegal oven bbq ribs,2282344,0
126693,222001,cheesy bacon bread,689540,5
126694,354979,kittencal s pan seared steak stove top to ove...,2000242659,5


In [183]:
df = reviews[['recipe_id']].value_counts()
df = df.rename_axis('recipe_id').reset_index()
df['review_count'] = df[0]
del df[0]

df = recipes[['name','id']].merge(df,
                            how = 'left',
                            left_on= 'id',
                            right_on='recipe_id',
                            ).fillna(0)
del df['recipe_id']

In [185]:
with pd.ExcelWriter("7.xlsx") as writer:
    df1.to_excel(writer, sheet_name="Рецепты с оценками")  
    df.to_excel(writer, sheet_name="Количество отзывов по рецептам") 